In [ ]:
from __future__ import print_function
import os
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
from custom_model.predictor import *
from custom_model.losses import *
from custom_model.datagen import *
from custom_model.inference import *
import numpy as np
import keras
from keras.optimizers import Adam
import os
from config import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.get_logger().setLevel('INFO')

import warnings
warnings.filterwarnings("ignore")
import time

In [ ]:
# run this section to train an ensemble of 15 beta-regression models
para['mode'] = 'spatialattention'
para['modelnorm'] = 'batch' #'layer'
para['epoches'] = 64
para['output_form'] = 'beta'
para['normalize'] = 1
para['batch_size'] = 8
para['adajacency_range'] = 5
para['nb_classes'] = 131
para['nb_blocks'] = 10
para['pred']=10
para['interval'] = 1
para['time_length'] = 5

BATCH_SIZE = para['batch_size']
EPOCH_NUMBER = para['epochs']
train_gen = DataGenerator(para, BATCH_SIZE, 'train')
val_gen = DataGenerator(para, BATCH_SIZE, 'val')

def scheduler(epoch):
   return 0.001*tf.math.exp(-epoch*0.07)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
for i in range(15):
    model = build_crossmodel(para)
    model.compile(loss = nll_beta(),
                  optimizer=Adam())

    history = model.fit(train_gen, validation_data = val_gen, validation_freq=3,
              epochs=30, callbacks=[callback], verbose='auto')
    model.save_weights('./DE/beta/model'+str(i)+'/model')

In [ ]:
# run this section to train several histogram-regression models
para['mode'] = 'dgc'
para['modelnorm'] = 'batch'
para['epoches'] = 64
para['output_form'] = 'histogram'
para['normalize'] = 1
para['batch_size'] = 8
para['adajacency_range'] = 5
para['nb_classes'] = 131
para['nb_blocks'] = 10
para['pred']=10
para['interval'] = 1
para['time_length'] = 5

BATCH_SIZE = para['batch_size']
EPOCH_NUMBER = para['epochs']
train_gen = DataGenerator(para, BATCH_SIZE, 'train')
val_gen = DataGenerator(para, BATCH_SIZE, 'val')

def scheduler(epoch):
   return 0.001*tf.math.exp(-epoch*0.085)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
for i in range(5):
    model = build_model(para)
    model.compile(loss = focal_loss(),
                  optimizer=Adam())

    history = model.fit(train_gen, validation_data = val_gen, validation_freq=3,
              epochs=30, callbacks=[callback], verbose='auto')
    model.save_weights('./DE/histogramf/model'+str(i)+'/model')

In [ ]:
# an example code to run the inference and save the parameters
test_gen = DataGenerator(para, BATCH_SIZE, 'testin')
A, B, Y = EnsembleInference(test_gen, para, nb_ensemble=10)
np.savez_compressed('./result/2019', A=A, B=B, Y=Y)